
1. Building a basic NL2SQL model
2. Adding few-shot examples
3. Dynamic few-shot example selection
4. Dynamic relevant table selection (Large DB Isssue)
5. Customizing prompts
6. Adding memory to the chatbot so that it answers follow-up questions related to the database.

https://www.mysqltutorial.org/getting-started-with-mysql/mysql-sample-database/

In [1]:
db_user = "root"
db_password = ""
db_host = "localhost"
db_name = "node-sql_test"

###Building a basic NL2SQL model

In [2]:
from langchain_community.utilities.sql_database import SQLDatabase
# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
db = SQLDatabase.from_uri("mysql+pymysql://root:@localhost:3307/node-sql_test")

In [3]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

mysql
['assignees', 'beers', 'tasks']

CREATE TABLE assignees (
	id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) NOT NULL, 
	email VARCHAR(320) NOT NULL, 
	`phoneNum` INTEGER(11) NOT NULL, 
	dob DATE NOT NULL, 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_general_ci ENGINE=InnoDB

/*
3 rows from assignees table:
id	name	email	phoneNum	dob
1	Aakash	aakash@example.com	123456789	1994-09-22
2	Prithvi	prithvi@example.com	121212121	1989-07-14
3	Jal	jal@example.com	82345438	2003-09-29
*/


CREATE TABLE beers (
	id BIGINT(20) NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) NOT NULL, 
	variety VARCHAR(255) NOT NULL, 
	description TEXT NOT NULL, 
	image VARCHAR(50) NOT NULL, 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_general_ci ENGINE=InnoDB

/*
3 rows from beers table:
id	name	variety	description	image
1	Buzz	djsbdkjn	dunk;lnsnknd 	nfnsdnknkndn
2	IDK	jdjsnnksf	lkdk	jdkskjsd
3	IDK2	jdjsnnf	kdk	jdd
*/


CREATE TABLE tasks (
	id INTEGER(11) NOT NULL AUTO_INC

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAIKEYHERE"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
LANGCHAIN_PROJECT = "demo"
os.environ["LANGCHAIN_API_KEY"] = "LangchainAPI"

In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "What is the dob of `Prithvi` in `assignees` ?"})
# "what is price of `1968 Ford Mustang`"
print(query)

```sql
SELECT `dob` FROM `assignees` WHERE `name` = 'Prithvi';
```


In [ ]:
chain = create_sql_query_chain(llm, db)
question2 = "Tell me the `tasks` assigned to `Prithvi`, their `start` and `end` dates from `tasks`?"

prompt2 = f"""You are a SQL assistant. Convert the following natural language question into an SQL query. Do not provide any explanation or answers, only return the SQL query. Do not change the table names or column names in the query and misinterpret the question.
Question: {question2}
SQL Query:
"""

# query2 = llm.invoke(prompt2)
# print(query2.content)
query2 = chain.invoke({"question": question2})
print(query2)

```sql
SELECT `name`, `start`, `end` 
FROM `tasks` 
WHERE `taskAssignee` = 'Prithvi' 
LIMIT 5;
```


In [33]:
import re

def clean_sql_query(text: str) -> str:
    """
    Clean SQL query by removing code block syntax, various SQL tags, backticks,
    prefixes, and unnecessary whitespace while preserving the core SQL query.

    Args:
        text (str): Raw SQL query text that may contain code blocks, tags, and backticks

    Returns:
        str: Cleaned SQL query
    """
    # Step 1: Remove code block syntax and any SQL-related tags
    # This handles variations like ```sql, ```SQL, ```SQLQuery, etc.
    block_pattern = r"```(?:sql|SQL|SQLQuery|mysql|postgresql)?\s*(.*?)\s*```"
    text = re.sub(block_pattern, r"\1", text, flags=re.DOTALL)

    # Step 2: Handle "SQLQuery:" prefix and similar variations
    # This will match patterns like "SQLQuery:", "SQL Query:", "MySQL:", etc.
    prefix_pattern = r"^(?:SQL\s*Query|SQLQuery|MySQL|PostgreSQL|SQL)\s*:\s*"
    text = re.sub(prefix_pattern, "", text, flags=re.IGNORECASE)

    # Step 3: Extract the first SQL statement if there's random text after it
    # Look for a complete SQL statement ending with semicolon
    sql_statement_pattern = r"(SELECT.*?;)"
    sql_match = re.search(sql_statement_pattern, text, flags=re.IGNORECASE | re.DOTALL)
    if sql_match:
        text = sql_match.group(1)

    # Step 4: Remove backticks around identifiers
    text = re.sub(r'`([^`]*)`', r'\1', text)

    # Step 5: Normalize whitespace
    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)

    # Step 6: Preserve newlines for main SQL keywords to maintain readability
    keywords = ['SELECT', 'FROM', 'WHERE', 'GROUP BY', 'HAVING', 'ORDER BY',
               'LIMIT', 'JOIN', 'LEFT JOIN', 'RIGHT JOIN', 'INNER JOIN',
               'OUTER JOIN', 'UNION', 'VALUES', 'INSERT', 'UPDATE', 'DELETE']

    # Case-insensitive replacement for keywords
    pattern = '|'.join(r'\b{}\b'.format(k) for k in keywords)
    text = re.sub(f'({pattern})', r'\n\1', text, flags=re.IGNORECASE)

    # Step 7: Final cleanup
    # Remove leading/trailing whitespace and extra newlines
    text = text.strip()
    text = re.sub(r'\n\s*\n', '\n', text)

    return text


In [34]:
print(clean_sql_query(query2))

SELECT name, start, end 
FROM tasks 
WHERE taskAssignee = 'Prithvi' 
LIMIT 5;


In [35]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)

'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near \'```sql\\nSELECT `dob` FROM `assignees` WHERE `name` = \'Prithvi\';\\n```\' at line 1")\n[SQL: ```sql\nSELECT `dob` FROM `assignees` WHERE `name` = \'Prithvi\';\n```]\n(Background on this error at: https://sqlalche.me/e/14/f405)'

In [37]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

chain = generate_query | RunnableLambda(clean_sql_query) | execute_query

# chain.invoke({"question": "What is the dob of `Prithvi` in `assignees` ?"})
chain.invoke({"question": "Tell me the `tasks` assigned to `Prithvi`, their `start` and `end` dates from `tasks`?"})

"[('Task 1', datetime.datetime(2025, 1, 26, 17, 45, 52), datetime.datetime(2025, 1, 28, 23, 22, 20)), ('Task 7', datetime.datetime(2025, 2, 18, 12, 36, 20), datetime.datetime(2025, 2, 19, 12, 36, 20))]"

In [39]:
chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [43]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, format the answer in a clean, structured manner.

User Question:
{question}  

SQL Query:
```sql
{query}
SQL Result:
{result}

Formatted Answer:
Please present the answer with all the details in a structured format and ensure that the response is clear and concise.

Answer: """ )

rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=generate_query | RunnableLambda(clean_sql_query)).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)

chain.invoke({"question": "Tell me the `tasks` assigned to `Prithvi`, their `start` and `end` dates from `tasks`?"})

'Here are the tasks assigned to Prithvi, along with their start and end dates:\n\n1. **Task Name:** Task 1\n   - **Start Date:** January 26, 2025, 17:45:52\n   - **End Date:** January 28, 2025, 23:22:20\n\n2. **Task Name:** Task 7\n   - **Start Date:** February 18, 2025, 12:36:20\n   - **End Date:** February 19, 2025, 12:36:20\n\nThese are the tasks currently assigned to Prithvi, limited to the first two entries.'

###Adding few-shot examples

In [ ]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
    {
        "input": "Show product details for products in the 'Motorcycles' product line.",
        "query": "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    {
        "input": "Retrieve the names of employees who report to employee number 1002.",
        "query": "SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;"
    },
    {
        "input": "List all products with a stock quantity less than 7000.",
        "query": "SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;"
    },
    {
     'input':"what is price of `1968 Ford Mustang`",
     "query": "SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;"
    }
]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # input_variables=["input","top_k"],
    input_variables=["input"],
)
print(few_shot_prompt.format(input1="How many products are there?"))

Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: Retrieve the names of employees who report to employee number 1002.
SQLQuery:
AI: SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: what is price of `1968 Ford Mustang`
SQLQuery:
AI: SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;


###Dynamic few-shot example selection

In [ ]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    vectorstore,
    k=2,
    input_keys=["input"],
)
example_selector.select_examples({"input": "how many employees we have?"})
# example_selector.select_examples({"input": "How many employees?"})

[{'input': 'Retrieve the names of employees who report to employee number 1002.',
  'query': 'SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;'},
 {'input': 'List all customers in France with a credit limit over 20,000.',
  'query': "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"}]

In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input="How many products are there?"))

Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';


Customizing prompts

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info"))

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: How many products are there?


In [ ]:
generate_query = create_sql_query_chain(llm, db,final_prompt)

chain = (
    RunnablePassthrough.assign(query=generate_query | RunnableLambda(clean_sql_query)).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)

chain.invoke({"question": "How many csutomers with credit limit more than 50000"})

'There are 85 customers with a credit limit more than 50,000.'

###Dynamic relevant table selection

In [ ]:
from operator import itemgetter
#from langchain.chains.openai_tools import create_extraction_chain_pydantic
from pydantic import BaseModel, Field
from typing import List
import pandas as pd

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("/content/database_table_descriptions.csv")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + "Table Name:" + row['table_name'] + "\n" + "Table Description:" + row['description'] + "\n\n"

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: List[str] = Field(description="List of Name of tables in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)

Table Name:productlines
Table Description:Stores information about the different product lines offered by the company, including a unique name, textual description, HTML description, and image. Categorizes products into different lines.

Table Name:products
Table Description:Contains details of each product sold by the company, including code, name, product line, scale, vendor, description, stock quantity, buy price, and MSRP. Linked to the productlines table.

Table Name:offices
Table Description:Holds data on the company's sales offices, including office code, city, phone number, address, state, country, postal code, and territory. Each office is uniquely identified by its office code.

Table Name:employees
Table Description:Stores information about employees, including number, last name, first name, job title, contact info, and office code. Links to offices and maps organizational structure through the reportsTo attribute.

Table Name:customers
Table Description:Captures data on cus

In [ ]:
table_details_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """Return the names of ALL the SQL tables that MIGHT be relevant to the user question.
                          The tables are:

                          {table_details}

                          Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""),
            ("human", "{question}")
        ]
    )

structured_llm = llm.with_structured_output(Table)

table_chain = table_details_prompt | structured_llm
tables = table_chain.invoke({"question": "give me details of customer and their order count", "table_details":table_details})
tables

Table(name=['customers', 'orders'])

In [ ]:
def get_tables(table_response: Table) -> List[str]:
    """
    Extracts the list of table names from a Table object.

    Args:
        table_response (Table): A Pydantic Table object containing table names.

    Returns:
        List[str]: A list of table names.
    """
    return table_response.name

select_table = {"question": itemgetter("question"), "table_details": itemgetter("table_details")} | table_chain | get_tables
select_table.invoke({"question": "give me details of customer and their order count", "table_details":table_details})

['customers', 'orders']

In [ ]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query | RunnableLambda(clean_sql_query)).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many cutomers with order count more than 5", "table_details":table_details})

'There are 2 customers with an order count of more than 5.'

In [ ]:
chain.invoke({"question": "Can you list their names?", "table_details":table_details})

"The names of the customers are:\n\n1. Atelier graphique\n2. La Rochelle Gifts\n3. Saveley & Henriot, Co.\n4. Daedalus Designs Imports\n5. La Corne D'abondance, Co.\n6. Mini Caravy\n7. Alpha Cognac\n8. Lyon Souveniers\n9. Auto Associés & Cie.\n10. Marseille Mini Autos\n11. Reims Collectables\n12. Auto Canal+ Petit"

###Adding memory to the chatbot so that it answers follow-up questions related to the database.






In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info",messages=[]))

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: How many products are there?


In [ ]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

generate_query = create_sql_query_chain(llm, db,final_prompt)

chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query | RunnableLambda(clean_sql_query)).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)


In [ ]:
question = "How many cutomers with order count more than 5"
response = chain.invoke({"question": question,"messages":history.messages, "table_details":table_details})
response

'There are 2 customers with an order count of more than 5.'

In [ ]:
history.add_user_message(question)
history.add_ai_message(response)


In [ ]:
history.messages

[HumanMessage(content='How many cutomers with order count more than 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='There are 2 customers with an order count of more than 5.', additional_kwargs={}, response_metadata={})]

In [1]:
response = chain.invoke({"question": "Can you list there names?","messages":history.messages, "table_details":table_details})
response

KeyboardInterrupt: 